# Noise Generation

## import libraries


In [ ]:
import numpy as np
import soundfile as sf
import os
import os
import pandas as pd
import torchaudio
from IPython.display import Audio
import matplotlib.pyplot as plt

In [ ]:
path = './test.wav'

# Load an example audio file
waveform, sample_rate = torchaudio.load(path)

# Play the audio
Audio(waveform.numpy(), rate=sample_rate)

In [ ]:
def plot_waveform(waveform):

    plt.figure(figsize=(10, 4))
    plt.plot(waveform[0])
    plt.title("Audio Signal")
    plt.xlabel("Sample Index")
    plt.ylabel("Amplitude")
    plt.show()

def plot_spectrogram(waveform, sample_rate):
    plt.figure(figsize=(10, 4))
    plt.specgram(waveform[0], Fs=sample_rate)
    plt.title("Spectrogram")
    plt.xlabel("Time")
    plt.ylabel("Frequency")
    plt.colorbar()
    plt.show()

In [ ]:
plot_waveform(waveform)
plot_spectrogram(waveform, sample_rate)
display(Audio(waveform.numpy(), rate=sample_rate))

In [ ]:
from numpy import ndarray


def add_scaled_noise(signal: ndarray, intensity=0.5, noise_type="gaussian") -> ndarray:
    """
    Dodaje szum o określonej intensywności względem sygnału.

    :param signal: Sygnał wejściowy.
    :param intensity: Intensywność szumu względem RMS sygnału (np. 0.5 dla 50%).
    :param noise_type: Typ szumu ('gaussian', 'uniform').
    :return: Sygnał z dodanym szumem.
    """
    rms_signal = np.sqrt(np.mean(signal**2))
    rms_noise = intensity * rms_signal

    match noise_type:
        case "gaussian":
            noise = np.random.normal(0, rms_noise, signal.shape)
        case "rayleigh":
            # Rayleigh noise requires scaling its parameter to achieve target RMS
            sigma = rms_noise / np.sqrt(2 - np.pi / 2)
            noise = np.random.rayleigh(sigma, signal.shape)
        case "poisson":
            # Scale Poisson noise approximation
            # Use the signal as lambda and scale it to match RMS intensity
            noise = np.random.poisson(rms_noise, signal.shape) - rms_signal
            noise = noise * rms_noise / np.sqrt(np.mean(noise**2))
        case _:
            raise ValueError("Nieobsługiwany typ szumu.")

    return signal + noise

In [ ]:
noisy_waveform = add_scaled_noise(waveform.numpy(), intensity=1, noise_type="gaussian")
display(Audio(noisy_waveform, rate=sample_rate))
plot_waveform(noisy_waveform)
plot_spectrogram(noisy_waveform, sample_rate)


In [ ]:
noisy_waveform = add_scaled_noise(waveform.numpy(), intensity=0.5, noise_type="gaussian")
display(Audio(noisy_waveform, rate=sample_rate))
plot_waveform(noisy_waveform)
plot_spectrogram(noisy_waveform, sample_rate)

In [ ]:
noisy_waveform = add_scaled_noise(waveform.numpy(), intensity=1, noise_type="rayleigh")
display(Audio(noisy_waveform, rate=sample_rate))
plot_waveform(noisy_waveform)
plot_spectrogram(noisy_waveform, sample_rate)

In [ ]:
noisy_waveform = add_scaled_noise(waveform.numpy(), intensity=0.5, noise_type="rayleigh")
display(Audio(noisy_waveform, rate=sample_rate))
plot_waveform(noisy_waveform)
plot_spectrogram(noisy_waveform, sample_rate)

In [ ]:
noisy_waveform = add_scaled_noise(waveform.numpy(), intensity=1, noise_type="poisson")
display(Audio(noisy_waveform, rate=sample_rate))
plot_waveform(noisy_waveform)
plot_spectrogram(noisy_waveform, sample_rate)

In [ ]:
noisy_waveform = add_scaled_noise(waveform.numpy(), intensity=0.5, noise_type="poisson")
display(Audio(noisy_waveform, rate=sample_rate))
plot_waveform(noisy_waveform)
plot_spectrogram(noisy_waveform, sample_rate)

## Sinusoidal waveform

In [ ]:
# make a sinusoidal signal
from sympy import plot


t = np.linspace(0, 1, 3200)  # Make the signal shorter to show only one loop
f = 20

signal = np.sin(2 * np.pi * f * t)

# play and show the signal
display(Audio(signal, rate=16000))

plt.figure(figsize=(10, 4))
plt.plot(signal)
plt.title("Sinusoidal Signal")
plt.xlabel("Sample Index")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
noisy_signal = add_scaled_noise(signal, intensity=1, noise_type="gaussian")
display(Audio(noisy_signal, rate=16000))
plt.figure(figsize=(10, 4))
plt.plot(noisy_signal)
plt.title("Sinusoidal Signal")
plt.xlabel("Sample Index")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
noisy_signal = add_scaled_noise(signal, intensity=1, noise_type="rayleigh")
display(Audio(noisy_signal, rate=16000))
plt.figure(figsize=(10, 4))
plt.plot(noisy_signal)
plt.title("Sinusoidal Signal")
plt.xlabel("Sample Index")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
noisy_signal = add_scaled_noise(signal, intensity=1, noise_type="poisson")
display(Audio(noisy_signal, rate=16000))
plt.figure(figsize=(10, 4))
plt.plot(noisy_signal)
plt.title("Sinusoidal Signal")
plt.xlabel("Sample Index")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
from numpy import ndarray


def add_noise(signal: ndarray, snr_db=20, noise_type="gaussian") -> ndarray:
    """
    Dodaje szum do sygnału.

    :param signal: Sygnał wejściowy.
    :param noise_type: Typ szumu ('gaussian', 'uniform', 'poisson', 'impulse').
    :param snr_db: Pożądane SNR w decybelach.
    :return: Sygnał z dodanym szumem.
    """
    signal_average_power = np.mean(signal**2)
    signal_average_power_db = 10 * np.log10(signal_average_power)
    noise_db = signal_average_power_db - snr_db
    noise_power = 10 ** (noise_db / 10)
    rms_noise = np.sqrt(noise_power)
    mean_noise = 0

    if noise_type == "gaussian":
        noise = np.random.normal(mean_noise, rms_noise, signal.shape)
    elif noise_type == "uniform":
        noise = np.random.uniform(-rms_noise, rms_noise, signal.shape)
    elif noise_type == "poisson":
        noise = np.random.poisson(rms_noise, signal.shape) - rms_noise
    else:
        raise ValueError("Nieobsługiwany typ szumu.")

    return signal + noise